In [ ]:
! unzip -o ../input/dogs-vs-cats-redux-kernels-edition/train.zip -d train > /dev/null
! unzip -o ../input/dogs-vs-cats-redux-kernels-edition/test.zip -d test > /dev/null

In [ ]:
import os
from pathlib import Path
from zipfile import ZipFile

import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split 
from sklearn.metrics import (
    accuracy_score, classification_report, 
    roc_curve,auc, plot_confusion_matrix,
    plot_precision_recall_curve
)

import matplotlib.pyplot as plt

def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

# Prep Data

Includes some small data augmentation

Splits the data into three sets, only augmenting the training data.

In [ ]:
# creating dataframe

# labels
file = os.listdir("./train/train")
Labels = list(map(lambda x: x.split('.')[0], file))

# for filenames
f = Path("./train/train")
File_Path = list(f.glob(r"**/*.jpg"))

# dataframe
File_Path = pd.Series(File_Path).astype(str)
Labels = pd.Series(Labels)
df = pd.concat([File_Path,Labels],axis=1)
df.columns = ['filename', 'category']

df.head()

In [ ]:
# Split data into train/val/hold
train_set, hold_data = train_test_split(df, test_size= 0.2, random_state = 1)
train_data, val_data = train_test_split(train_set, test_size= 0.4, random_state = 2)

batch_size = 32

train_size = train_data.shape[0]
train_steps_per_epoch = math.ceil(train_size/batch_size)

val_size = val_data.shape[0]
val_steps_per_epoch = math.ceil(val_size/batch_size)

hold_size = hold_data.shape[0]
hold_steps_per_epoch = math.ceil(hold_size/batch_size)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=30,
        brightness_range=[0.5,1.5],
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_dataframe(
    train_data, 
    x_col='filename',
    y_col='category',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=32,
    shuffle = True
)

validation_generator = test_datagen.flow_from_dataframe(
    val_data, 
    x_col='filename',
    y_col='category',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=32,
    shuffle = False)

hold_generator = test_datagen.flow_from_dataframe(
    hold_data, 
    x_col='filename',
    y_col='category',
    target_size=(150, 150),
    class_mode='binary',
    batch_size=32,
    shuffle = False)


# Pretrained Model

[Details of model being used for transfer learning](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/)

In [ ]:
pretrained = tf.keras.applications.EfficientNetB5(
    include_top=False,
    weights="imagenet"
)

pretrained.trainable = False

In [ ]:
transfered_model = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Resizing(456, 456),
        pretrained,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy', threshold=0.5),
           tf.keras.metrics.Recall(name = 'recall'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.AUC(name = 'auc')
           ]

transfered_model.compile(
        loss="binary_crossentropy",
        metrics=metrics,
        optimizer=tf.keras.optimizers.Adam(0.001)
    )

Train for a few epochs with only the classifer changing. Keep the pretrained weights frozen.

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

earlystop = EarlyStopping(patience=6)


result = transfered_model.fit(
    train_generator, 
    validation_data=validation_generator,
    epochs = 10,
    steps_per_epoch = train_steps_per_epoch,
    validation_steps = val_steps_per_epoch, 
    callbacks = [earlystop, learning_rate_reduction]
) 

plot_hist(result)

Train for a few more epochs with the full set up weights training

In [ ]:
for layer in transfered_model.layers[1].layers:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

transfered_model.compile(
        loss="binary_crossentropy",
        metrics=metrics,
        optimizer=tf.keras.optimizers.Adam(0.00001)
    )

result = transfered_model.fit(
    train_generator, 
    validation_data=validation_generator,
    epochs = 10,
    steps_per_epoch = train_steps_per_epoch,
    validation_steps = val_steps_per_epoch, 
    callbacks = [earlystop, learning_rate_reduction]
) 

plot_hist(result)

# Check Out Validation

In [ ]:
# Predict the label of the test_images
p = transfered_model.predict(hold_generator).reshape((-1,))
label = (p > 0.5).astype(int)
truth = (hold_data.category == 'dog').astype(int)

In [ ]:
IC = type('IdentityClassifier', (), 
          {"predict": lambda i : i, 
           "predict_proba": lambda i : i, 
           "_estimator_type": "classifier", 
           "classes_": ['Cat', 'Dog']})
plot_confusion_matrix(IC, truth, label);

In [ ]:
import matplotlib.pyplot as plt

fpr, tpr, threshold = roc_curve(truth, p)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', 
         label = 'AUC = %0.2f' % roc_auc,
         linewidth=4.0)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
print(classification_report(truth, label))

# Out of Sample Prediction

In [ ]:
test_filenames = os.listdir("./test/test")
test_df = pd.DataFrame({
    'filename': test_filenames
})

test_gen = test_datagen.flow_from_dataframe(
    test_df, 
    "./test/test",
    x_col='filename',
    y_col=None,
    target_size=(150, 150),
    class_mode=None,
    batch_size=32,
    shuffle = False # not to shuffle the given data
)

In [ ]:
%%time

pred = pd.Series(transfered_model.predict(test_gen).reshape((-1,)))
index = pd.Series([int(x.split('.')[0]) for x in test_df.filename])

submission = pd.DataFrame({'id':index, 'label':pred})

submission.to_csv('submission.csv', index=False)